# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
**_Opening a New Gym Center in Singapore**
- Build a dataframe of neighborhoods in Singapore by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new Gym Center
***
### 1. Import libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 98 kB 8.8 MB/s  eta 0:00:01
     |████████████████████████████████| 94 kB 3.5 MB/s eta 0:00:011
Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [3]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Places_in_Singapore").text

In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [5]:
# create a list to store neighborhood data
neighborhoodList = []

In [6]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[1].findAll("li"):
    neighborhoodList.append(row.text)

In [7]:
# create a new DataFrame from the list
sg_df = pd.DataFrame({"Neighborhood": neighborhoodList})

sg_df.head()

,Neighborhood
0,List of commercial sites in Singapore
1,List of places in Singapore
2,"Admiralty, Singapore"
3,Airport Logistics Park
4,Alkaff Gardens


In [8]:
# print the number of rows of the dataframe
sg_df.shape

(191, 1)

### 3. Get the geographical coordinates

In [9]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Singapore'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in sg_df["Neighborhood"].tolist() ]

In [11]:
coords

[[1.30215000000004, 103.86365000000006],
 [1.2904100000000653, 103.85211000000004],
 [1.4573513836356915, 103.81818372540098],
 [1.378020000000049, 104.00095000000005],
 [1.295220000000029, 103.85877000000005],
 [1.3716100000000324, 103.84546000000006],
 [1.3850100000000225, 103.84510000000006],
 [1.3023270067125625, 103.85813792658826],
 [1.3242500000000632, 103.95297000000005],
 [1.3327000000000453, 103.93714000000006],
 [1.3204607814059668, 103.86513527905811],
 [1.3195806402641834, 103.68613925693634],
 [1.295220000000029, 103.85877000000005],
 [1.3118300000000431, 103.79545000000007],
 [1.3507900000000745, 103.85110000000009],
 [1.31628967233386, 103.86329353979332],
 [1.3333300000000463, 103.70000000000005],
 [1.2904100000000653, 103.85211000000004],
 [1.38005674846275, 103.8918232554782],
 [1.3008100000000695, 103.85591000000005],
 [1.349520000000041, 103.75277000000006],
 [1.3482673252931419, 103.75171721711418],
 [1.2904100000000653, 103.85211000000004],
 [1.3641700000000583, 

In [12]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [13]:
# merge the coordinates into the original dataframe
sg_df['Latitude'] = df_coords['Latitude']
sg_df['Longitude'] = df_coords['Longitude']

In [14]:
# check the neighborhoods and the coordinates
print(sg_df.shape)
sg_df

(191, 3)


,Neighborhood,Latitude,Longitude
0,List of commercial sites in Singapore,1.302150,103.863650
1,List of places in Singapore,1.290410,103.852110
2,"Admiralty, Singapore",1.457351,103.818184
3,Airport Logistics Park,1.378020,104.000950
4,Alkaff Gardens,1.295220,103.858770
5,Ang Mo Kio,1.371610,103.845460
6,Ang Mo Kio Police Division,1.385010,103.845100
7,Arab Street,1.302327,103.858138
8,Bedok,1.324250,103.952970
9,Bedok Police Division,1.332700,103.937140


In [15]:
# save the DataFrame as CSV file
sg_df.to_csv("sg_df.csv", index=False)

### 4. Create a map of Singapore with neighborhoods superimposed on top

In [16]:
# get the coordinates of Singapore
address = 'Singapore'

geolocator = Nominatim(user_agent="ny-explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore 1.357107, 103.8194992.


In [17]:
# create map of Singapore using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(sg_df['Latitude'], sg_df['Longitude'], sg_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_sg)  
    
map_sg

In [18]:
# save the map as HTML file
map_sg.save('map_sg.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [19]:
# define Foursquare Credentials and Version
CLIENT_ID = 'RY5RKWDUYRZS50JJI3XX3BJ3PJPKZUMPIC34NQF42Q1F1VUC' # your Foursquare ID
CLIENT_SECRET = 'XGF4K35X4QQCXMZNHDC100O1CVBTGMOGOC1XJXARVQ3NVF3L' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RY5RKWDUYRZS50JJI3XX3BJ3PJPKZUMPIC34NQF42Q1F1VUC
CLIENT_SECRET:XGF4K35X4QQCXMZNHDC100O1CVBTGMOGOC1XJXARVQ3NVF3L


**Now, let's get the top 100 venues that are within a radius of 1000 meters.**

In [116]:
radius = 1000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(sg_df['Latitude'], sg_df['Longitude'], sg_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [117]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(12306, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,List of commercial sites in Singapore,1.30215,103.86365,The Projector,1.302255,103.863553,Multiplex
1,List of commercial sites in Singapore,1.30215,103.86365,Wakey Wakey,1.301162,103.861926,Coffee Shop
2,List of commercial sites in Singapore,1.30215,103.86365,Sari Ratu Restaurant,1.302145,103.861258,Indonesian Restaurant
3,List of commercial sites in Singapore,1.30215,103.86365,Mrs Pho,1.301323,103.861880,Vietnamese Restaurant
4,List of commercial sites in Singapore,1.30215,103.86365,PizzaFace,1.301028,103.861535,Italian Restaurant


**Let's check how many venues were returned for each neighorhood**

In [118]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Admiralty, Singapore",21,21,21,21,21,21
Airport Logistics Park,15,15,15,15,15,15
Alkaff Gardens,100,100,100,100,100,100
Ang Mo Kio,80,80,80,80,80,80
Ang Mo Kio Police Division,27,27,27,27,27,27
Arab Street,100,100,100,100,100,100
Bedok,34,34,34,34,34,34
Bedok Police Division,68,68,68,68,68,68
"Bendemeer, Singapore",51,51,51,51,51,51


**Let's find out how many unique categories can be curated from all the returned venues**

In [119]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 355 uniques categories.


In [120]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Multiplex', 'Coffee Shop', 'Indonesian Restaurant',
       'Vietnamese Restaurant', 'Italian Restaurant', 'Thai Restaurant',
       'Burger Joint', 'Cafeteria', 'Hotel', 'BBQ Joint', 'Art Gallery',
       'Café', 'Food Court', 'Restaurant', 'French Restaurant', 'Bakery',
       'Yoga Studio', 'Sandwich Place', 'Tea Room',
       'Taiwanese Restaurant', 'Cocktail Bar', 'Indian Restaurant',
       'Pool', 'Hookah Bar', 'Bed & Breakfast', 'Asian Restaurant',
       'Whisky Bar', 'Spa', 'Park', 'Beer Bar', 'Japanese Restaurant',
       'Bar', 'Jazz Club', 'Vegetarian / Vegan Restaurant',
       'New American Restaurant', 'Wine Bar', 'Mediterranean Restaurant',
       'Mexican Restaurant', 'Movie Theater', 'Racetrack', 'Supermarket',
       'Halal Restaurant', 'Bridge', 'Dessert Shop', 'Chinese Restaurant',
       'Shopping Mall', 'Gym / Fitness Center', 'Ramen Restaurant',
       'Dim Sum Restaurant', 'Arts & Crafts Store'], dtype=object)

In [121]:
# check if the results contain "Gym/Fitness Center"
"Gym / Fitness Center" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [122]:
# one hot encoding
sg_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sg_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sg_onehot.columns[-1]] + list(sg_onehot.columns[:-1])
sg_onehot = sg_onehot[fixed_columns]

print(sg_onehot.shape)
sg_onehot.head()

(12306, 356)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cable Car,Cafeteria,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Casino,Cemetery,Cha Chaan Teng,Cheese Shop,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Gym,College Lab,College Tennis Court,College Theater,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Lighthouse,Lottery Retailer,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaur

**Count the number of Gym/Fitness centers for each neighborhood**

In [124]:
gym_venues_df = sg_onehot.loc[sg_onehot["Gym / Fitness Center"]>0, ["Neighborhoods", "Gym / Fitness Center"]]
gym_count = gym_venues_df.groupby(["Neighborhoods"])["Gym / Fitness Center"].count().reset_index()
gym_count.rename(columns={"Neighborhoods": "Neighborhood", "Gym / Fitness Center": "No. of Gym/Fitness Center"}, inplace=True)
print(gym_count.shape)
gym_count.head()

(68, 2)


,Neighborhood,No. of Gym/Fitness Center
0,Alkaff Gardens,2
1,Ang Mo Kio,1
2,Ang Mo Kio Police Division,1
3,Arab Street,1
4,Bidadari Garden,2


**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [125]:
sg_grouped = sg_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(sg_grouped.shape)
sg_grouped

(191, 356)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cable Car,Cafeteria,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Casino,Cemetery,Cha Chaan Teng,Cheese Shop,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Gym,College Lab,College Tennis Court,College Theater,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Lighthouse,Lottery Retailer,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaur

In [126]:
len(sg_grouped[sg_grouped["Gym / Fitness Center"] > 0])

68

**Create a new DataFrame for Gym/Fitness Center data only**

In [127]:
sg_gym = sg_grouped[["Neighborhoods","Gym / Fitness Center"]]

In [134]:
sg_gym.head()
sg_gym.shape

(191, 2)

### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Singapore into 3 clusters.

In [136]:
# set number of clusters
kclusters = 3

sg_clustering = sg_gym.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sg_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

191

In [151]:
# create a new dataframe that includes the cluster.
sg_merged = sg_gym.copy()

# add clustering labels
sg_merged["Cluster Labels"] = kmeans.labels_
sg_merged.shape

(191, 3)

In [152]:
sg_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
sg_merged.head()
sg_merged.shape

(191, 3)

In [153]:
# merge sg_merged with gym_count and sg_df to add number of gym centers and latitude/longitude for each neighborhood
sg_merged = pd.merge(sg_merged, gym_count, on="Neighborhood", how="left")
sg_merged["No. of Gym/Fitness Center"] = sg_merged["No. of Gym/Fitness Center"].replace(np.nan, 0)
sg_merged = sg_merged.join(sg_df.set_index("Neighborhood"), on="Neighborhood")

print(sg_merged.shape)
sg_merged.head()

(191, 6)


,Neighborhood,Gym / Fitness Center,Cluster Labels,No. of Gym/Fitness Center,Latitude,Longitude
0,"Admiralty, Singapore",0.000000,0,0.0,1.457351,103.818184
1,Airport Logistics Park,0.000000,0,0.0,1.378020,104.000950
2,Alkaff Gardens,0.020000,2,2.0,1.295220,103.858770
3,Ang Mo Kio,0.012500,2,1.0,1.371610,103.845460
4,Ang Mo Kio Police Division,0.037037,1,1.0,1.385010,103.845100


In [154]:
# sort the results by Cluster Labels
print(sg_merged.shape)
sg_merged.sort_values(["Cluster Labels"], inplace=True)
sg_merged

(191, 6)


,Neighborhood,Gym / Fitness Center,Cluster Labels,No. of Gym/Fitness Center,Latitude,Longitude
0,"Admiralty, Singapore",0.000000,0,0.0,1.457351,103.818184
91,"Kembangan, Singapore",0.000000,0,0.0,1.444033,103.825493
90,Kebun Baru,0.000000,0,0.0,1.370710,103.837080
89,Katong,0.000000,0,0.0,1.304570,103.902880
88,Kampong Ubi,0.000000,0,0.0,1.316670,103.900000
144,Sembawang,0.000000,0,0.0,1.447940,103.818910
86,Kampong Glam,0.000000,0,0.0,1.304130,103.863470
85,Kampong Bugis,0.000000,0,0.0,1.346940,103.965830
145,Sembawang Shopping Centre,0.000000,0,0.0,1.441770,103.824500
83,Kallang Bahru,0.000000,0,0.0,1.319758,103.869249


**Finally, let's visualize the resulting clusters**

In [155]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sg_merged['Latitude'], sg_merged['Longitude'], sg_merged['Neighborhood'], sg_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [156]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [161]:
cluster_0 = sg_merged.loc[sg_merged['Cluster Labels'] == 0]
print(cluster_0.shape[0])
cluster_0

123


,Neighborhood,Gym / Fitness Center,Cluster Labels,No. of Gym/Fitness Center,Latitude,Longitude
0,"Admiralty, Singapore",0.0,0,0.0,1.457351,103.818184
91,"Kembangan, Singapore",0.0,0,0.0,1.444033,103.825493
90,Kebun Baru,0.0,0,0.0,1.370710,103.837080
89,Katong,0.0,0,0.0,1.304570,103.902880
88,Kampong Ubi,0.0,0,0.0,1.316670,103.900000
144,Sembawang,0.0,0,0.0,1.447940,103.818910
86,Kampong Glam,0.0,0,0.0,1.304130,103.863470
85,Kampong Bugis,0.0,0,0.0,1.346940,103.965830
145,Sembawang Shopping Centre,0.0,0,0.0,1.441770,103.824500
83,Kallang Bahru,0.0,0,0.0,1.319758,103.869249


#### Cluster 1

In [162]:
cluster_1 = sg_merged.loc[sg_merged['Cluster Labels'] == 1]
print(cluster_1.shape[0])
cluster_1

8


,Neighborhood,Gym / Fitness Center,Cluster Labels,No. of Gym/Fitness Center,Latitude,Longitude
182,"Western Islands, Singapore",0.062500,1,1.0,1.331100,103.695510
166,Telok Ayer Street,0.050000,1,5.0,1.280960,103.847752
186,Yew Tee,0.071429,1,2.0,1.394440,103.753890
190,"Yunnan, Singapore",0.043478,1,1.0,1.337348,103.690630
93,Kian Teck,0.055556,1,1.0,1.332837,103.695545
31,"Central Area, Singapore",0.040000,1,4.0,1.288830,103.846250
4,Ang Mo Kio Police Division,0.037037,1,1.0,1.385010,103.845100
14,Boon Lay,0.042553,1,2.0,1.333330,103.700000


#### Cluster 2

In [163]:
cluster_2 = sg_merged.loc[sg_merged['Cluster Labels'] == 2]
print(cluster_2.shape[0])
cluster_2

60


,Neighborhood,Gym / Fitness Center,Cluster Labels,No. of Gym/Fitness Center,Latitude,Longitude
127,Pasir Panjang,0.029412,2,1.0,1.292290,103.768190
32,Central Police Division,0.030000,2,3.0,1.278930,103.839090
169,Template:Punggol,0.016393,2,1.0,1.402460,103.906860
168,Template:Places in Singapore,0.020000,2,2.0,1.290410,103.852110
36,Choa Chu Kang,0.027778,2,1.0,1.386160,103.746180
37,"City Hall, Singapore",0.010000,2,1.0,1.292570,103.856870
40,"Coleman House, Singapore",0.010000,2,1.0,1.292080,103.850393
164,Tanjong Rhu,0.016129,2,1.0,1.297160,103.876250
41,"Commonwealth, Singapore",0.010526,2,1.0,1.304798,103.797610
44,Dhoby Ghaut,0.010000,2,1.0,1.298740,103.848000


#### Observations:
There are 123 out of the 191 neighborhoods fall into cluster 0 which showing as red circle on the map. Cluster 0 provides a catergory of very low frequency of occurance of Gym and Fitness Center in the neighborhoods. In fact, there is zero existance of Gym and Fitness Centers in these areas. 60 neighborhoods in green in cluster 2 is considered as as moderate level of frequency of occurance. The balance of 8 neighborhoods in cluster 1 as purple shows the highest frequency of occurance of Gym and Fitness Center in the neighborhoods.